In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import cv2
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Model
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from keras.models import load_model

Using TensorFlow backend.


In [2]:
#file paths
dataset_path = 'storage/SebastianRosales/Trashnet/dataset/dataset-resized'
dataset_classes = ['cardboard','glass','metal','paper','plastic','trash']
dataset_classes_encoded = [0,1,2,3,4,5]

In [3]:
#image convertiong to rgb array
def convert(image_path, label):
    image = img_to_array(load_img(image_path, target_size=[299, 299]))
    image = image * float(1/255.0) #pre-process.
    return image, label

def convert_inverted(image_path, label):
    image = img_to_array(load_img(image_path, target_size=[299, 299]))
    image = cv2.bitwise_not(image)
    image = image * float(1/255.0) #pre-process.
    return image, label

def convert_flipped(image_path, label):
    image = img_to_array(load_img(image_path, target_size=[299, 299]))
    image = cv2.flip(image, -1)
    image = image * float(1/255.0) #pre-process.
    return image, label

def convert_flipped_inverted(image_path, label):
    image = img_to_array(load_img(image_path, target_size=[299, 299]))
    image = cv2.bitwise_not(image)
    image = cv2.flip(image, -1)
    image = image * float(1/255.0) #pre-process.
    return image, label

#read the data and pre-process it
def read_data(data_path, dataset_classes,dataset_classes_encoded):

    training_labels = tf.keras.utils.to_categorical(dataset_classes_encoded, num_classes = 6)

    labels = []
    data = []
    
    for each_class in dataset_classes:
        directory = data_path + '/' + each_class
        for filename in os.listdir(directory):
            if filename.endswith('.jpg'):
                label = training_labels[dataset_classes.index(each_class)]
                aux_1, aux_2 = convert(directory + '/'+ filename,label)
                labels.append(aux_2)
                data.append(aux_1)
                #aux_3, aux_4 = convert_inverted(directory + '/'+ filename,label)
                #labels.append(aux_4)
                #data.append(aux_3)
                aux_5, aux_6 = convert_flipped(directory + '/'+ filename,label)
                labels.append(aux_6)
                data.append(aux_5)
                aux_7, aux_8 = convert_flipped_inverted(directory + '/'+ filename,label)
                labels.append(aux_8)
                data.append(aux_7)
    
    labels = np.asarray(labels) 
    data = np.asarray(data)
    return data, labels

In [4]:
inputs, labels = read_data(dataset_path, dataset_classes, dataset_classes_encoded)

In [5]:
epochs = 100
batch_size = 32
X_train, X_dummy, Y_train, Y_dummy = train_test_split(inputs, labels, test_size = 0.3, random_state = 2)

In [6]:
X_test, X_val, Y_test, Y_val = train_test_split(X_dummy, Y_dummy, test_size = 0.4, random_state = 2)

In [7]:
input_shape = inputs[0].shape
output_class = 6

In [8]:
model = InceptionResNetV2(weights='imagenet')
model.layers.pop()
model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    
#model.summary()

In [9]:
new_input_data = model.predict(X_train)
new_input_data_test = model.predict(X_test)

In [34]:
model_from_scratch = Sequential()
model_from_scratch.add(Dense(new_input_data.shape[0], input_dim=new_input_data.shape[1], activation='relu'))
model_from_scratch.add(Dense(48, activation='relu'))
model_from_scratch.add(Dense(12, activation='relu'))
model_from_scratch.add(Dense(output_class, activation='softmax'))
model_from_scratch.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 5306)              8155322   
_________________________________________________________________
dense_32 (Dense)             (None, 48)                254736    
_________________________________________________________________
dense_33 (Dense)             (None, 12)                588       
_________________________________________________________________
dense_34 (Dense)             (None, 6)                 78        
Total params: 8,410,724
Trainable params: 8,410,724
Non-trainable params: 0
_________________________________________________________________


In [35]:
model_from_scratch.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

In [36]:
history = model_from_scratch.fit(new_input_data, Y_train,validation_data=(new_input_data_test, Y_test),
                       epochs=epochs, batch_size=batch_size, verbose=1,
                       callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')])

Train on 5306 samples, validate on 1365 samples
Epoch 1/100
5306/5306 [==============================] - 2s 374us/step - loss: 1.4060 - accuracy: 0.4361 - val_loss: 1.1676 - val_accuracy: 0.5590
Epoch 2/100
5306/5306 [==============================] - 2s 344us/step - loss: 0.9875 - accuracy: 0.6221 - val_loss: 0.9297 - val_accuracy: 0.6447
Epoch 3/100
5306/5306 [==============================] - 2s 345us/step - loss: 0.8760 - accuracy: 0.6530 - val_loss: 0.8907 - val_accuracy: 0.6454
Epoch 4/100
5306/5306 [==============================] - 2s 346us/step - loss: 0.8182 - accuracy: 0.6766 - val_loss: 0.9745 - val_accuracy: 0.6615
Epoch 5/100
5306/5306 [==============================] - 2s 350us/step - loss: 0.7563 - accuracy: 0.7051 - val_loss: 0.8338 - val_accuracy: 0.6696
Epoch 6/100
5306/5306 [==============================] - 2s 345us/step - loss: 0.7113 - accuracy: 0.7147 - val_loss: 0.8385 - val_accuracy: 0.6637
Epoch 7/100
5306/5306 [==============================] - 2s 340us/step

In [37]:
score = model_from_scratch.evaluate(new_input_data_test, Y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.7324491916122017 / Test accuracy: 0.7465201616287231


In [38]:
validation = model.predict(X_val)

In [39]:
score = model_from_scratch.evaluate(validation, Y_val, verbose=0)
print(f'Validation loss: {score[0]} / Validation accuracy: {score[1]}')

Validation loss: 0.739300557414254 / Validation accuracy: 0.7351648211479187


In [40]:
model_from_scratch.save('Thrasnet-InceptionResNetV2_adadelta.h5')

# xception below

In [41]:
model = Xception(weights='imagenet')
model.layers.pop()
model = Model(inputs=model.inputs, outputs=model.layers[-1].output)

#model.summary()

In [42]:
new_input_data = model.predict(X_train)
new_input_data_test = model.predict(X_test)

In [43]:
model_from_scratch = Sequential()
model_from_scratch.add(Dense(new_input_data.shape[0], input_dim=new_input_data.shape[1], activation='relu'))
model_from_scratch.add(Dense(48, activation='relu'))
model_from_scratch.add(Dense(12, activation='relu'))
model_from_scratch.add(Dense(output_class, activation='softmax'))
#model_from_scratch.summary()

In [44]:
model_from_scratch.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
history = model_from_scratch.fit(new_input_data, Y_train,validation_data=(new_input_data_test, Y_test),
                       epochs=epochs, batch_size=batch_size, verbose=1,
                       callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')])

Train on 5306 samples, validate on 1365 samples
Epoch 1/100
5306/5306 [==============================] - 2s 458us/step - loss: 1.0985 - accuracy: 0.5754 - val_loss: 0.9090 - val_accuracy: 0.6381
Epoch 2/100
5306/5306 [==============================] - 2s 412us/step - loss: 0.7900 - accuracy: 0.6937 - val_loss: 0.9157 - val_accuracy: 0.6381
Epoch 3/100
5306/5306 [==============================] - 2s 414us/step - loss: 0.6842 - accuracy: 0.7314 - val_loss: 0.7687 - val_accuracy: 0.6960
Epoch 4/100
5306/5306 [==============================] - 2s 412us/step - loss: 0.6049 - accuracy: 0.7605 - val_loss: 0.7733 - val_accuracy: 0.7048
Epoch 5/100
5306/5306 [==============================] - 2s 420us/step - loss: 0.5555 - accuracy: 0.7787 - val_loss: 0.7148 - val_accuracy: 0.7319
Epoch 6/100
5306/5306 [==============================] - 2s 417us/step - loss: 0.5167 - accuracy: 0.7910 - val_loss: 0.7484 - val_accuracy: 0.7143
Epoch 7/100
5306/5306 [==============================] - 2s 414us/step

In [46]:
score = model_from_scratch.evaluate(new_input_data_test, Y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.8541932220424051 / Test accuracy: 0.7443223595619202


In [47]:
validation = model.predict(X_val)

In [48]:
score = model_from_scratch.evaluate(validation, Y_val, verbose=0)
print(f'Validation loss: {score[0]} / Validation accuracy: {score[1]}')

Validation loss: 0.7915817332791758 / Validation accuracy: 0.7615384459495544


In [49]:
model_from_scratch.save('Thrasnet-xception_adadelta.h5')